In [1]:
import hashlib
from cryptography.fernet import Fernet
def hash_password(password):
    password = password.encode()
    hashed = hashlib.sha256(password).hexdigest()
    return hashed


In [2]:
key = Fernet.generate_key()
f = Fernet(key)
def encrypt(data):
    token = f.encrypt(data)
    return token

def decrypt(token):
    text = f.decrypt(token)
    return text

In [3]:
def get_valid_password():
    ok = False
    while ok == False:
        pw = input("Enter a password : ")
        score = check_strength(pw)
        print("Strength score =", score)

        if score >= 6:
            ok = True
        else:
            print("Password too weak, try again")

    return pw


In [4]:
def save_user_credentials(username, hashed_pw):
    with open("user_names.txt","a") as file:
        file.write(username + "\n")

    with open("passwords.txt","a") as file2:
        file2.write(hashed_pw + "\n")


In [5]:
def check_user(Uname):
    Uname = str(Uname)
    with open("user_names.txt",'r') as file:
        lines = file.readlines()

    for line in lines:
        if Uname == line.strip():
            return True

    return False


In [6]:
def login():
    Uname = input("Username: ")
    Pw = input("Password: ")

    hashed = hash_password(Pw)

    with open("user_names.txt","r") as uf:
        users = uf.readlines()

    with open("passwords.txt","r") as pf:
        pwds = pf.readlines()

    for i in range(len(users)):
        if users[i].strip() == Uname and pwds[i].strip() == hashed:
            return True

    return False


In [7]:
import os

def make_user_folder(username):
    folder = "user_data_" + username
    if os.path.exists(folder) == False:
        os.makedirs(folder)
    return folder


In [8]:
def save_encrypted_file(username, fname, token):
    folder = make_user_folder(username)
    path = folder + "/" + fname

    with open(path,"wb") as file:
        file.write(token)


In [9]:
def load_and_decrypt(username, fname):
    folder = "user_data_" + username
    path = folder + "/" + fname

    with open(path,"rb") as file:
        token = file.read()

    text = decrypt(token)
    return text


In [10]:
def delete_file(username, fname):
    folder = "user_data_" + username
    path = folder + "/" + fname

    if os.path.exists(path):
        os.remove(path)
        print("File deleted")
    else:
        print("File not found")


In [11]:
def list_files(username):
    folder = "user_data_" + username

    if os.path.exists(folder) == False:
        print("No files found")
        return

    files = os.listdir(folder)
    for f in files:
        print(f)


In [12]:
def add_user():
    print("_____ ADDING NEW USER _____")
    Uname = input("Choose a username: ")

    # check if user already exists
    if check_user(Uname) == True:
        print("User already exists")
        return

    # get a strong password
    print("Create a strong password")
    pw = get_valid_password()

    hashed_pw = hash_password(pw)

    # save username and hashed password
    save_user_credentials(Uname, hashed_pw)

    # make user folder
    make_user_folder(Uname)

    print("User created")


In [13]:
def upload_file(username):
    print("_____ UPLOAD FILE _____")
    fname = input("Enter file name to store: ")
    text = input("Enter file contents: ")

    text = text.encode()
    token = encrypt(text)

    save_encrypted_file(username, fname, token)

    print("File encrypted and saved")


In [14]:
def view_file(username):
    print("_____ VIEW FILE _____")
    fname = input("Enter file name: ")

    try:
        data = load_and_decrypt(username, fname)
        print("Decrypted text:")
        print(data.decode())
    except:
        print("Could not open or decrypt file")


In [15]:
def delete_user_file(username):
    print("_____ DELETE FILE _____")
    fname = input("Enter file name: ")
    delete_file(username, fname)


In [16]:
def show_files(username):
    print("_____ USER FILES _____")
    list_files(username)


In [17]:
def user_menu(username):
    print("Welcome", username)

    run = True
    while run == True:
        print("")
        print("Choose an option:")
        print("1. Upload file")
        print("2. View file")
        print("3. Delete file")
        print("4. List files")
        print("5. Logout")

        choice = input("Enter choice: ")

        if choice == "1":
            upload_file(username)
        elif choice == "2":
            view_file(username)
        elif choice == "3":
            delete_user_file(username)
        elif choice == "4":
            show_files(username)
        elif choice == "5":
            run = False
        else:
            print("Invalid choice")


In [18]:
def check_strength(password):
    lower_letters = "abcdefghijklmnopqrstuvwxyz"
    upper_letters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    digits = "0123456789"
    symbols = "~@#$%^&*()_+-=,./:;|"

    has_lower = False
    has_upper = False
    has_digit = False
    has_symbol = False

    repeated_chars = False
    has_sequence = False

    common_bad = ["password", "qwerty", "abc123", "111111", "123456"]

    for i in range(len(password)):
        ch = password[i]

        if ch in lower_letters:
            has_lower = True
        if ch in upper_letters:
            has_upper = True
        if ch in digits:
            has_digit = True
        if ch in symbols:
            has_symbol = True

        if i > 0 and password[i] == password[i-1]:
            repeated_chars = True

        if i >= 2:
            a = password[i-2]
            b = password[i-1]
            c = password[i]

            if a in lower_letters and b in lower_letters and c in lower_letters:
                if ord(b) == ord(a)+1 and ord(c) == ord(b)+1:
                    has_sequence = True

            if a in upper_letters and b in upper_letters and c in upper_letters:
                if ord(b) == ord(a)+1 and ord(c) == ord(b)+1:
                    has_sequence = True

            if a.isdigit() and b.isdigit() and c.isdigit():
                if int(b) == int(a)+1 and int(c) == int(b)+1:
                    has_sequence = True

    score = 0

    if len(password) >= 8:
        score += 2
    if len(password) >= 12:
        score += 1

    if has_lower: score += 2
    if has_upper: score += 2
    if has_digit: score += 2
    if has_symbol: score += 2

    if repeated_chars:
        score -= 1
    if has_sequence:
        score -= 2

    pw_lower = password.lower()
    for bad in common_bad:
        if bad in pw_lower:
            score -= 3

    if score < 0:
        score = 0
    if score > 10:
        score = 10

    return score

In [19]:
def main_menu():
    run = True
    while run == True:
        print("\n_____ MAIN MENU _____")
        print("1. Register")
        print("2. Login")
        print("3. Exit")

        choice = input("Enter choice: ")

        if choice == "1":
            add_user()
        elif choice == "2":
            print("_____ LOGIN _____")
            ok = login()
            if ok == True:
                uname = input("Enter username again to continue: ")
                user_menu(uname)
            else:
                print("Login failed")
        elif choice == "3":
            run = False
        else:
            print("Invalid choice")


In [ ]:
main_menu()


_____ MAIN MENU _____
1. Register
2. Login
3. Exit


Enter choice:  1


_____ ADDING NEW USER _____


Choose a username:  mumer


Create a strong password


...

Decrypted text:
paise ghari gaari

Choose an option:
1. Upload file
2. View file
3. Delete file
4. List files
5. Logout


Enter choice:  5



_____ MAIN MENU _____
1. Register
2. Login
3. Exit


Enter choice:  2


_____ LOGIN _____
